In [40]:
from google.colab import files
uploaded = files.upload()

Saving Data_RDM Websites.xlsx to Data_RDM Websites.xlsx


In [120]:
import io
import numpy as np
import pandas as pd

df = pd.read_excel(io.BytesIO(uploaded['Data_RDM Websites.xlsx']), sheet_name="final")
#df

In [114]:
# Code adapted from https://www.mediawiki.org/wiki/API:Geosearch#Python
import requests

# Function to get the coordinates from an entity for which there are coordinates recorded in the Wikipedia Webpage
# Note (to work, the name of the entity must be exactly the same as the name of the entity in Wikipedia)
def get_coordinates (entity, endpoint):

  S = requests.Session()
  URL = endpoint

  lat, longi = None, None

  PARAMS = {
      "action": "query",
      "format": "json",
      "titles": entity,
      "prop": "coordinates"
  }

  R = S.get(url=URL, params=PARAMS)
  DATA = R.json()
  PAGES = DATA['query']['pages']

  for k, v in PAGES.items():
      print(v)
      if v.get('coordinates', None) is not None:
        lat = v['coordinates'][0]['lat']
        longi = v['coordinates'][0]['lon']

  return [lat, longi]

# Test the function just created
entity = "Freie Universität Berlin"
endpoint = "https://de.wikipedia.org/w/api.php"

print(get_coordinates(entity, endpoint))

{'pageid': 15042, 'ns': 0, 'title': 'Freie Universität Berlin', 'coordinates': [{'lat': 52.45305556, 'lon': 13.29055556, 'primary': '', 'globe': 'earth'}]}
[52.45305556, 13.29055556]


In [ ]:
# add location information to all institutions
locations = [get_coordinates(e, endpoint) for e in df['Wikipedia Name']]
df['location'] = locations


In [118]:
df

,Consortium,Partner,Wikipedia Name,RDM Website,location
0,NFDI4Earth,Brandenburgische Technische Universität Cottbu...,Brandenburgische Technische Universität Cottbu...,https://www.b-tu.de/en/bibliothek/publishing/r...,"[51.76722222, 14.32638889]"
1,NFDI4Earth,Fachinformationsdienst Geowissenschaften des G...,Helmholtz-Zentrum Potsdam – Deutsches GeoForsc...,https://www.fidgeo.de/en/research-data/faq,"[52.38255556, 13.06444444]"
2,NFDI4Earth,Freie Universität Berlin,Freie Universität Berlin,https://www.fu-berlin.de/en/sites/forschungsda...,"[52.45305556, 13.29055556]"
3,NFDI4Earth,GEOMAR Helmholtz-Zentrum für Ozeanforschung,Helmholtz-Zentrum für Ozeanforschung Kiel,https://portal.geomar.de/home,"[54.32723, 10.18185]"
4,NFDI4Earth,Helmholtz-Zentrum Potsdam – Deutsches GeoForsc...,Helmholtz-Zentrum Potsdam – Deutsches GeoForsc...,https://dataservices.gfz-potsdam.de/portal/drr...,"[52.38255556, 13.06444444]"
...,...,...,...,...,...
68,NFDI4Microbiodata,Universität zu Köln,Universität zu Köln,https://fdm.uni-koeln.de/en/rdm-knowledge/faq,"[50.92805556, 6.92861111]"
69,NFDI4Microbiodata,Universität Hamburg,Universität Hamburg,https://www.fdm.uni-hamburg.de/en/fdm/faq.html,"[53.566944, 9.983889]"
70,NFDI4Microbiodata,Friedrich-Schiller-Universität Jena,Friedrich-Schiller-Universität Jena,https://www.researchdata.uni-jena.de/en/faq,"[50.92944444, 11.58944444]"
71,NFDI4Microbiodata,Christian-Albrechts-Universität zu Kiel,Christian-Albrechts-Universität zu Kiel,https://www.datamanagement.uni-kiel.de/en/rese...,"[54.33888889, 10.1225]"


In [116]:
# Export the dataset as json
import json

# skip the 'wikipedia name' column for the creation of the json file
df2 = df.drop(labels='Wikipedia Name', axis=1)

data = json.dumps(json.loads(df2.to_json(orient="records")), ensure_ascii=False)
data

'[{"Consortium": "NFDI4Earth", "Partner": "Brandenburgische Technische Universität Cottbus-Senftenberg", "RDM Website": "https://www.b-tu.de/en/bibliothek/publishing/research-data", "location": [51.76722222, 14.32638889]}, {"Consortium": "NFDI4Earth", "Partner": "Fachinformationsdienst Geowissenschaften des GeoForschungsZentrum Potsdam", "RDM Website": "https://www.fidgeo.de/en/research-data/faq", "location": [52.38255556, 13.06444444]}, {"Consortium": "NFDI4Earth", "Partner": "Freie Universität Berlin", "RDM Website": "https://www.fu-berlin.de/en/sites/forschungsdatenmanagement/glossar/index.html", "location": [52.45305556, 13.29055556]}, {"Consortium": "NFDI4Earth", "Partner": "GEOMAR Helmholtz-Zentrum für Ozeanforschung", "RDM Website": "https://portal.geomar.de/home", "location": [54.32723, 10.18185]}, {"Consortium": "NFDI4Earth", "Partner": "Helmholtz-Zentrum Potsdam – Deutsches GeoForschungsZentrum", "RDM Website": "https://dataservices.gfz-potsdam.de/portal/drr.html", "location"

Added Manually


*   [TU Berlin](https://en.wikipedia.org/wiki/Technical_University_of_Berlin); 52.511944, 13.326389; German Wikipedia page existing, but no coordinates available. Coordinates picked from the Wikipedia page in English.
*   [Leibniz-Zentrum für Agrarlandschaftsforschung (ZALF)](https://de.wikipedia.org/wiki/Leibniz-Zentrum_f%C3%BCr_Agrarlandschaftsforschung); 52.51588285, 14.112973799647975; German Wikipedia page existing, but no coordinates available.
*  [Martin-Luther-Universität-Halle-Wittenberg](https://de.wikipedia.org/wiki/Martin-Luther-Universit%C3%A4t_Halle-Wittenberg); 51.486389, 11.968889; German Wikipedia page existing, but no coordinates available. Coordinates picked from the Wikipedia page in English.
* [Georg-August-Universität Göttingen, Campus-Institut Data Science (CIDAS)](https://goettingen-campus.de/de/cidas); 51.53410245, 9.937839941393724; No wikipedia page, different location from University of Göttingen.
* [Institut für Angewandte Informatik (InfAI)](https://de.wikipedia.org/wiki/Institut_f%C3%BCr_Angewandte_Informatik); 51.3419872,12.3684174 ; German Wikipedia page existing, but no coordinates available.
* [Helmholtz-Institut für Funktionelle Marine Biodiversität an der Universität Oldenburg (HIFMB)](https://de.wikipedia.org/wiki/Helmholtz-Institut_f%C3%BCr_Funktionelle_Marine_Biodiversit%C3%A4t_an_der_Universit%C3%A4t_Oldenburg); 53.1532481, 8.1715979; German Wikipedia page existing, but no coordinates available.

The geocoding was done using [this](https://geocode.maps.co/) api. For an example (ZALF), see [here](https://geocode.maps.co/search?q={Eberswalder%20Stra%C3%9Fe%2084,%2015374%20M%C3%BCncheberg}).